# Code for <br> Vincent 2015: A tutorial on Bayesian Models of Perception

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from notebooks.imports import *


In [ ]:
import pymc as pm
import arviz as az
import pytensor
import scipy

# setting float precision in pytensor
pytensor.config.floatX = "float32"
import jax
jax.config.update("jax_enable_x64", False)
# check if GPU is available
import torch
if torch.cuda.is_available():
    print(f"Currently using: {torch.cuda.get_device_name(0)} device")
else:
    print("No GPU available, using CPU instead.")

### Load Configs

In [4]:
from config import dir_config, main_config

raw_dir = Path(dir_config.data.raw)

### Utils functions

In [5]:
import scipy.special


def m1_posterior_prediction(T, si, variance):
    """
    Generate random samples of k based on the given parameters.

    Parameters:
    - T: Number of trials
    - si: Stimulus intensities (scalar or array)
    - variance: Variance parameter (scalar)

    Returns:
    - k: Random samples of k based on the generated binomial distribution
    - PCc: Probability of correct response (scalar or array)
    """

    # Calculate PC using normal CDF
    PCc = scipy.stats.norm.cdf(si / np.sqrt(2 * variance))
    
    # Sample from binomial distribution
    k = np.random.binomial(T, PCc)

    return k, PCc


#### True Model Parameters

In [6]:
true_variance = 1
true_T = 100
true_si = np.logspace(-2, 2, 10)
true_bias = 0
true_lapse_rate = 0.01
true_prior = np.array([0.5, 0.5])

### Model 1: Statistical Decision Theory with Bayesian estimates

#### 1. Simulate dataset

In [ ]:
np.random.seed(42)  # For reproducibility
data = {
    "sioriginal": true_si, # example stimulus intensity
    "T": true_T, # number of trials
    "muN": 0, # mean of the prior
    "v": true_variance, # standard deviation of the prior
}
data["ni"] = 41 # number of iterations
data["sii"] =  np.logspace(-2, 2, data["ni"]) #  interpolated stimulus intensities
data["si"] = np.concatenate([data['sioriginal'], data['sii']]) 
# Generate data['koriginal'] by simulating counts
data["koriginal"], _ = m1_posterior_prediction(data['T'], data['sioriginal'], data['v'])

plt.plot(data['sioriginal'], data['koriginal'], 'o')


#### 2. Parameter Recovery

In [ ]:
# define the model and calculate the joint probability and posterior
with pm.Model() as model1:
    # Prior
    variance = pm.Uniform("variance", lower=0, upper=1000)
    
    # PCc using Normal CDF
    sioriginal = pm.Data("sioriginal", data["sioriginal"])
    PCc = pm.Deterministic("PCc", pm.math.invlogit(sioriginal / pm.math.sqrt(2 * variance)))
    T = pm.Data("T", data["T"])
    
    # likelihood
    kc = pm.Binomial("kc", n=T, p=PCc, observed=data["koriginal"])
    
model1.to_graphviz()


In [ ]:
with model1:
    trace = pm.sample(draws=1000, tune=500, chains=2, cores=4, target_accept=0.95, compute_convergence_checks=True)

In [ ]:
# Extract samples from the trace
variance_samples = trace.posterior["variance"].values.flatten()
# Plotting posterior distributions
plt.figure(figsize=(6, 6))
plt.hist(variance_samples, bins=30, density=True, alpha=0.7)
plt.title('Posterior Distribution of Variance')
plt.xlabel('Variance')
plt.ylabel('Density')


In [ ]:

# Extract the variance samples from the trace
variance_samples = trace.posterior["variance"].values.flatten()

# Generate simulated data using posterior samples
def generate_simulated_data(T, si, variance_samples):
    simulated_k = np.zeros((len(variance_samples), len(si)), dtype=int)
    
    for i, variance in enumerate(variance_samples):
        k, _ = m1_posterior_prediction(T, si, variance)
        simulated_k[i] = k
        
    return simulated_k.mean(axis=0)

simulated_k = generate_simulated_data(data['T'], data['si'], variance_samples)

# Plotting simulated data
plt.figure(figsize=(8, 6))
plt.plot(data['si'], simulated_k, marker='o', linestyle='', color='b', label='Simulated Data')
plt.plot(data['sioriginal'], data['koriginal'], marker='o', linestyle="", color='r', label='Original Data')
plt.title('Simulated Data')
plt.xlabel('Stimulus Intensity')
plt.ylabel('Number of Correct Responses')
plt.grid(True)
plt.legend()
plt.show()

## Model 2: A Trial-to-trial SDT model

#### 1. Simulate Data

In [ ]:
data["lr"] = true_lapse_rate
data["bias"] = true_bias
data["prior"] = true_prior
data["si"] = np.concatenate([data["sioriginal"], data["sioriginal"], data["sii"]])

In [ ]:
data["sioriginal"]

#### 2. Model

In [ ]:
import pytensor.tensor as tt
import pymc as pm

# Assuming 'data' is defined elsewhere in your script
# and it contains 'prior', 'si', and 'koriginal'

with pm.Model() as model2:
    # Prior
    bias = pm.Normal("bias", mu=0, sigma=1/2000)
    lapse_rate = pm.Beta("lapse_rate", alpha=1, beta=1)
    variance = pm.Uniform("variance", lower=0, upper=1000)
    target_location = pm.Categorical("prior_probability", p=data["prior"])
    
    # Likelihoods
    mu_x_perceived = pm.math.switch(tt.eq(target_location, 1), data["sioriginal"], 0)
    x_perceived = pm.Normal("x_perceived", mu=mu_x_perceived, sigma=1/variance)
    
    # Compute mct with compatible shapes
    condition = x_perceived - bias > 0  # condition should be a boolean array
    p_true = tt.stack([1 - lapse_rate/2, lapse_rate/2])  # shape (2,)
    p_false = tt.stack([lapse_rate/2, 1 - lapse_rate/2])  # shape (2,)
    
    # Ensure condition is reshaped correctly for broadcasting
    condition_broadcasted = tt.shape_padleft(condition)
    
    # Ensure p_true and p_false are reshaped to match condition_broadcasted
    p_true_broadcasted = tt.shape_padright(p_true, 1)
    p_false_broadcasted = tt.shape_padright(p_false, 1)
    
    # Use the switch function with the reshaped condition and p_true, p_false
    mct = pm.Deterministic('mct', pm.math.switch(condition_broadcasted, p_true_broadcasted, p_false_broadcasted))
    
    # Response posterior
    response = pm.Categorical("response", p=mct, observed=data["koriginal"])

model2.to_graphviz()
with model2:
    trace = pm.sample(draws=1000, tune=500, chains=2, cores=4, target_accept=0.95, compute_convergence_checks=False)